# Symbolic results 

It is also of interest to work with non-defined variables in order to look for analytical results. This can be done integrating the Fermionic package together with SymPy via PyCall. There are also packages for defining symbolic fermionic operators. In this notebook we will see how exactly does this work.

1. [Symbolic interaction](#symbol)
2. [Symbolic operators](#sops)


<a id="symbol"></a>
## Symbolic Algebra

In this section we will learn to interact with the symbolic tools defined in [SymPy](https://github.com/JuliaPy/SymPy.jl). This library works perfectly within Julia via PyCall. I will show you how:

First of all, if you have not done ir yet, you should install Conda:

`using Pkg
Pkg.add("Conda") 
using Conda
Conda.update()
Pkg.add("PyCall")`

We then call PyCall

In [1]:
using PyCall
sympy = pyimport("sympy");

With this amazing package we can work with symbols as they were variables

In [2]:
x = sympy.Symbol("x")      # PyObject x

PyObject x

In [3]:
sympy.Matrix([x, 2, 3])

PyObject Matrix([
[x],
[2],
[3]])

So, for instance, we can exactly diagonalize the superconducting hamiltonian in a symbolic way:
$H=\sum_k \epsilon_k (c_k^\dagger c_k + c_{\bar{k}}^\dagger c_{\bar{k}})-\sum_{k,k'} G_{k,k'} c_{k'}^\dagger c_{\bar{k'}}^\dagger c_{\bar{k}} c_k$
We will set $\epsilon_k = \epsilon \; \forall k$, and $G_{kk'} = g \; \forall k,k'$.

In [4]:
using Fermionic
using SparseArrays

g = sympy.Symbol("g")
e = sympy.Symbol("e")

o = Op(4); #this is for dimension 4
e0 = 1.0 #the energy spectrum of the free term
d = dim(o) 
epsilon = [e0*(i-d/4-1/2) for i in 1:d/2]
epsilon = sort([epsilon; epsilon])

h0 = sum([epsilon[i]*(cdm(o,i)*cm(o,i) + cdm(o,i+1)*cm(o,i+1)) for i in 1:2:(Int(d)-1)])

hi = sum([sum([if i==j spzeros(2^d, 2^d) else (cdm(o,j)*cdm(o,j+1)*cm(o,i+1)*cm(o,i)) end
                    for i in 1:2:(Int(d)-1)]) for j in 1:2:(Int(d)-1)]);
    
mh0 =  Matrix(h0);
mhi = Matrix(hi)

mh = e*sympy.Matrix(mh0) - g*sympy.Matrix(mhi)
(P, D) = mh.diagonalize();
D #this outputs the diagonal matrix.

PyObject Matrix([
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0, -0.5*e,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0, -0.5*e,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0

The eigenvalues are the columns of the following matrix:

In [5]:
P

PyObject Matrix([
[1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,                          0,                          0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,                          0,                          0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,                          0,                          0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, g/(e + (e**2 + g**2)**0.5), g/(e - (e**2 + g**2)**0.5)],
[  0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,                          0,                          0],
[  0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,                          0,                          0],
[  0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,                          0,                          0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,                

You can learn more about matrix manipulation in [this tutorial](https://docs.sympy.org/latest/modules/matrices/matrices.html).

<a id="sops"></a>
## Symbolic second quantization

There is a library inside SymPy which allows the definition of fermionic symbolic operators. You can finde the complete list of functions in [this link](https://docs.sympy.org/latest/modules/physics/secondquant.html).

It is good enough for working with commutators, but for some reasons the anticommutator identity is not defined for fermions, making it a little limited for complete usage.

In [6]:
sp = pyimport("sympy");
sq = pyimport("sympy.physics.secondquant");
adi = sq.Dagger(sq.F(sp.Symbol("i")));
adj = sq.Dagger(sq.F(sp.Symbol("j")))

PyObject CreateFermion(j)

In [7]:
ak = sq.Dagger(sq.Fd(sp.Symbol("k")))

PyObject AnnihilateFermion(k)

We can evauate contractions:

In [8]:
sq.contraction(adi,adi) #double creation operator

PyObject 0

In [9]:
sq.contraction(ak,ak) #double destruction operator

PyObject 0

In [10]:
a = sq.contraction(adi,ak)

PyObject KroneckerDelta(_i, k)*KroneckerDelta(i, k)

In [11]:
sq.evaluate_deltas(a)

PyObject KroneckerDelta(_i, i)

We can define commutators

In [12]:
com = sq.Commutator(adi*adj,ak)

PyObject Commutator(CreateFermion(i)*CreateFermion(j),f(k))

and expand them with doit()

In [13]:
com.doit(wicks = true) #wicked defines a normal ordering of terms (see Wick's expantion)

PyObject -KroneckerDelta(_a, i)*KroneckerDelta(i, k)*CreateFermion(j) + KroneckerDelta(_a, j)*KroneckerDelta(j, k)*CreateFermion(i) - KroneckerDelta(_i, k)*KroneckerDelta(i, k)*CreateFermion(j) + KroneckerDelta(_i, k)*KroneckerDelta(j, k)*CreateFermion(i)

Also perform a wick expantion

In [14]:
sq.wicks(adi*(ak+adj))

PyObject KroneckerDelta(_i, k)*KroneckerDelta(i, k) + NO(CreateFermion(i)*AnnihilateFermion(k)) + NO(CreateFermion(i)*CreateFermion(j))

In [15]:
sq.wicks(adi*(ak+adj), keep_only_fully_contracted = true)

PyObject KroneckerDelta(_i, k)*KroneckerDelta(i, k)

NO() performs a normal order, assuming anticommutation relations

In [16]:
sq.NO(ak*adi)

PyObject -NO(CreateFermion(i)*AnnihilateFermion(k))

In [17]:
adj = sq.Dagger(sq.CreateFermion(sp.Symbol("j")))

PyObject AnnihilateFermion(j)

Each state can only have one particle, so we choose to store a list of occupied orbits rather than a tuple with occupation numbers (zeros and ones).

In [18]:
f1 = sq.FKet([1,2])

PyObject FockStateKet((1, 2))

In [19]:
f2 = sq.FBra([1,3])

PyObject FockStateKet((1, 3))

In [20]:
sq.InnerProduct(f2,f1)

PyObject 0